<a href="https://colab.research.google.com/github/rsm-wew068/hack-mvp/blob/main/Music_AI_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 AI Music Recommendation System - Google Colab

This notebook runs the complete Music AI Recommendation System with GPT-OSS-20B and neural networks.

## 🚀 Features:
- AI-powered music recommendations
- GPT-OSS-20B natural language explanations
- Spotify integration with OAuth
- RLHF preference learning
- Interactive Streamlit frontend

## 📋 Setup Instructions:
1. **Enable GPU**: Runtime > Change runtime type > GPU (T4 or better)
2. **Run all cells** in order
3. **Get Spotify API credentials** from https://developer.spotify.com/dashboard
4. **Update credentials** in the config cell
5. **Enjoy your AI music recommendations!**

---


## 🔧 Step 1: Install Dependencies and Setup


In [ ]:
# Install required packages
%pip install -q streamlit fastapi uvicorn[standard] pydantic pydantic-settings
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers vllm scikit-learn numpy pandas
%pip install -q spotipy sqlalchemy aiosqlite httpx aiohttp requests
%pip install -q plotly rich python-dotenv google-colab
%pip install -q pyngrok

print("✅ Dependencies installed successfully!")


In [ ]:
# Clone the repository
!git clone https://github.com/rsm-wew068/hack-mvp.git
%cd hack-mvp

# Create necessary directories
!mkdir -p models data logs

print("✅ Repository cloned and directories created!")


In [ ]:
# Create necessary directories
!mkdir -p models data logs

print("✅ Repository cloned and directories created!")


In [ ]:
# Setup ngrok for Colab port forwarding (alternative method)
from pyngrok import ngrok
import getpass

# Get ngrok authtoken (optional but recommended)
print("🔗 Setting up ngrok for better port forwarding...")
print("   You can get a free ngrok authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken")
print("   (Optional - the app will work without it, but ngrok provides better URLs)")

# Uncomment and add your ngrok authtoken if you have one:
# ngrok.set_auth_token("your_ngrok_authtoken_here")

print("✅ ngrok setup complete!")


In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 GPU detected: {gpu_name} (Count: {gpu_count})")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected. Please enable GPU in Runtime > Change runtime type > GPU")
    print("   The system will still work but with slower performance")


In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 GPU detected: {gpu_name} (Count: {gpu_count})")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected. Please enable GPU in Runtime > Change runtime type > GPU")
    print("   The system will still work but with slower performance")


## 🔐 Step 2: Configure Spotify API Credentials


In [ ]:
# Configure Spotify API credentials
import os

# Get your credentials from https://developer.spotify.com/dashboard
# 1. Create a new app
# 2. Add redirect URI: https://colab.research.google.com/callback
# 3. Copy Client ID and Client Secret here

SPOTIFY_CLIENT_ID = "your_client_id_here"  # Replace with your actual Client ID
SPOTIFY_CLIENT_SECRET = "your_client_secret_here"  # Replace with your actual Client Secret

# Set environment variables
os.environ["SPOTIFY_CLIENT_ID"] = SPOTIFY_CLIENT_ID
os.environ["SPOTIFY_CLIENT_SECRET"] = SPOTIFY_CLIENT_SECRET
os.environ["SPOTIFY_REDIRECT_URI"] = "https://colab.research.google.com/callback"

# Colab-specific settings
os.environ["PYTHONPATH"] = "/content/hack-mvp"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["VLLM_HOST"] = "0.0.0.0"
os.environ["VLLM_PORT"] = "8002"
os.environ["API_BASE_URL"] = "http://localhost:8000"
os.environ["FRONTEND_URL"] = "http://localhost:8501"
os.environ["DATABASE_URL"] = "sqlite:///./data/music_app.db"
os.environ["MODEL_STORAGE_PATH"] = "./models"
os.environ["DATA_STORAGE_PATH"] = "./data"

print("✅ Environment configured!")
print(f"📱 Spotify Client ID: {SPOTIFY_CLIENT_ID[:10]}..." if SPOTIFY_CLIENT_ID != "your_client_id_here" else "⚠️  Please update your Spotify credentials above")


## 🧠 Step 3: Initialize AI Models


In [ ]:
# Initialize neural network models
import torch
import sys
sys.path.append('/content/hack-mvp')

from models.neural_networks import MusicEmbeddingNet, BradleyTerryModel, DeepCollaborativeFilter

print("🧠 Initializing neural network models...")

# Create models
audio_model = MusicEmbeddingNet(num_audio_features=13, embedding_dim=128)
bt_model = BradleyTerryModel(num_items=100000, embedding_dim=64)
cf_model = DeepCollaborativeFilter(num_users=10000, num_items=100000, embedding_dim=128)

# Save models
torch.save(audio_model.state_dict(), 'models/audio_embedding.pth')
torch.save(bt_model.state_dict(), 'models/bradley_terry.pth')
torch.save(cf_model.state_dict(), 'models/collaborative_filter.pth')

print("✅ Neural network models initialized and saved!")
print(f"📊 Model sizes:")
print(f"   Audio Embedding: {sum(p.numel() for p in audio_model.parameters()):,} parameters")
print(f"   Bradley-Terry: {sum(p.numel() for p in bt_model.parameters()):,} parameters")
print(f"   Collaborative Filter: {sum(p.numel() for p in cf_model.parameters()):,} parameters")


## 🚀 Step 4: Start All Services


In [ ]:
# Start all services in background
import subprocess
import threading
import time
import requests
from google.colab import output

print("🚀 Starting Music AI Recommendation System...")

# Enable Colab port forwarding
output.enable_custom_widget_manager()

def start_vllm_server():
    """Start vLLM server with GPT-OSS-20B."""
    print("🧠 Starting vLLM server with GPT-OSS-20B...")
    cmd = [
        "python", "-m", "vllm.entrypoints.api_server",
        "--model", "openai-community/gpt-oss-20b",
        "--host", "0.0.0.0",
        "--port", "8002",
        "--gpu-memory-utilization", "0.8",
        "--max-model-len", "4096"
    ]
    subprocess.run(cmd, capture_output=True)

def start_backend():
    """Start FastAPI backend."""
    print("⚡ Starting FastAPI backend...")
    cmd = [
        "python", "-m", "uvicorn",
        "backend.main:app",
        "--host", "0.0.0.0",
        "--port", "8000"
    ]
    subprocess.run(cmd, capture_output=True)

def start_frontend():
    """Start Streamlit frontend."""
    print("🖥️  Starting Streamlit frontend...")
    cmd = [
        "python", "-m", "streamlit", "run",
        "frontend/app.py",
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true"
    ]
    subprocess.run(cmd, capture_output=True)

# Start services in background threads
vllm_thread = threading.Thread(target=start_vllm_server, daemon=True)
backend_thread = threading.Thread(target=start_backend, daemon=True)
frontend_thread = threading.Thread(target=start_frontend, daemon=True)

vllm_thread.start()
backend_thread.start()
frontend_thread.start()

print("⏳ Services starting... This may take a few minutes for the first run.")
print("   - vLLM server: Downloading GPT-OSS-20B model (~2-3 minutes)")
print("   - Backend: Starting FastAPI server")
print("   - Frontend: Starting Streamlit app")


## ⏳ Step 5: Wait for Services to Start


In [ ]:
# 🔧 FIX FOR COLAB CONNECTION ISSUES
# If you're getting "localhost refused to connect" errors, run this cell

import subprocess
import time
from pyngrok import ngrok

print("🔧 Fixing Colab connection issues...")

# Method 1: Use ngrok for better port forwarding
try:
    print("🔗 Setting up ngrok tunnels...")

    # Start services first (if not already running)
    print("🚀 Starting services...")

    # Start backend
    backend_process = subprocess.Popen([
        "python", "-m", "uvicorn", "backend.main:app",
        "--host", "0.0.0.0", "--port", "8000"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Start frontend
    frontend_process = subprocess.Popen([
        "python", "-m", "streamlit", "run", "frontend/app.py",
        "--server.port", "8501", "--server.address", "0.0.0.0", "--server.headless", "true"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Wait a bit for services to start
    time.sleep(10)

    # Create ngrok tunnels
    frontend_tunnel = ngrok.connect(8501)
    backend_tunnel = ngrok.connect(8000)

    print("✅ Services started with ngrok tunnels!")
    print(f"🎵 Frontend (Streamlit): {frontend_tunnel.public_url}")
    print(f"⚡ Backend (API): {backend_tunnel.public_url}")
    print(f"📚 API Docs: {backend_tunnel.public_url}/docs")

    # Store URLs for later use
    globals()['frontend_url'] = frontend_tunnel.public_url
    globals()['backend_url'] = backend_tunnel.public_url

except Exception as e:
    print(f"❌ ngrok setup failed: {e}")
    print("🔄 Trying alternative method...")

    # Method 2: Use Colab's built-in port forwarding
    from google.colab import output
    output.enable_custom_widget_manager()

    print("📱 Using Colab's built-in port forwarding...")
    print("   Frontend: http://localhost:8501")
    print("   Backend: http://localhost:8000")
    print("   API Docs: http://localhost:8000/docs")

    # Try to open the app directly
    try:
        output.serve_kernel_port_as_window(8501, path='/', anchor_text='🎵 Open Music AI App')
        print("✅ Click the link above to open the app!")
    except:
        print("⚠️  Could not create direct link. Use the URLs above.")

print("\\n🎉 Connection fix complete!")


In [ ]:
# 🚨 TROUBLESHOOTING: If you still can't connect

print("🚨 Troubleshooting Connection Issues")
print("=" * 50)

# Check if services are running
import subprocess
import requests

def check_service(port, name):
    try:
        response = requests.get(f"http://localhost:{port}/health", timeout=5)
        if response.status_code == 200:
            print(f"✅ {name} (port {port}): Running")
            return True
        else:
            print(f"⚠️  {name} (port {port}): Status {response.status_code}")
            return False
    except:
        print(f"❌ {name} (port {port}): Not responding")
        return False

# Check all services
print("🔍 Checking service status:")
backend_ok = check_service(8000, "Backend API")
frontend_ok = check_service(8501, "Streamlit Frontend")

if not backend_ok or not frontend_ok:
    print("\\n🔧 Services not running. Let's restart them...")

    # Kill any existing processes
    subprocess.run(["pkill", "-f", "streamlit"], capture_output=True)
    subprocess.run(["pkill", "-f", "uvicorn"], capture_output=True)

    print("🔄 Restarting services...")

    # Start backend
    subprocess.Popen([
        "python", "-m", "uvicorn", "backend.main:app",
        "--host", "0.0.0.0", "--port", "8000"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Start frontend
    subprocess.Popen([
        "python", "-m", "streamlit", "run", "frontend/app.py",
        "--server.port", "8501", "--server.address", "0.0.0.0", "--server.headless", "true"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print("⏳ Waiting for services to start...")
    time.sleep(15)

    # Check again
    print("\\n🔍 Rechecking service status:")
    check_service(8000, "Backend API")
    check_service(8501, "Streamlit Frontend")

print("\\n💡 If you still can't connect:")
print("   1. Make sure you're using the ngrok URLs (if available)")
print("   2. Try refreshing the page")
print("   3. Check that your Spotify credentials are set correctly")
print("   4. Restart the Colab runtime and run all cells again")


In [ ]:
# Wait for services to be ready and test them
import time
import requests

print("⏳ Waiting for services to start...")
print("   This may take 3-5 minutes for the first run (model download)")

# Wait for initial startup
time.sleep(30)

# Test services
def test_service(url, name, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code == 200:
            print(f"✅ {name}: Ready (Status: {response.status_code})")
            return True
        else:
            print(f"⚠️  {name}: Status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"⏳ {name}: Still starting... ({str(e)[:50]}...)")
        return False

# Test all services
services = [
    ("http://localhost:8000/health", "Backend API"),
    ("http://localhost:8002/health", "vLLM Server"),
    ("http://localhost:8501/_stcore/health", "Streamlit Frontend")
]

all_ready = False
attempts = 0
max_attempts = 20

while not all_ready and attempts < max_attempts:
    attempts += 1
    print(f"\n🔄 Attempt {attempts}/{max_attempts}:")

    results = []
    for url, name in services:
        results.append(test_service(url, name))

    all_ready = all(results)

    if not all_ready:
        print(f"⏳ Waiting 15 seconds before next attempt...")
        time.sleep(15)

if all_ready:
    print("\n🎉 All services are ready!")
else:
    print("\n⚠️  Some services may still be starting. You can continue or restart the services.")


## 🎵 Step 6: Access Your Music AI App


In [ ]:
# Display access information
from IPython.display import HTML, display

print("🎉 Music AI Recommendation System is now running!")
print("\n📱 Access your application:")
print("\n🖥️  Frontend (Streamlit App):")
print("   The Streamlit app should be embedded below or accessible through Colab's interface")

# Try to embed the Streamlit app
try:
    from google.colab import output
    output.serve_kernel_port_as_iframe(8501, path='/', anchor_text='🎵 Open Music AI App')
    print("   ✅ Click the link above to open the app!")
except:
    print("   📱 App is running on http://localhost:8501")

print("\n⚡ Backend API:")
print("   http://localhost:8000")
print("\n📚 API Documentation:")
print("   http://localhost:8000/docs")

print("\n🧠 vLLM Server (GPT-OSS-20B):")
print("   http://localhost:8002")

print("\n🎯 What you can do:")
print("   1. 🔐 Authenticate with Spotify")
print("   2. 🎵 Get AI-powered music recommendations")
print("   3. 🎯 Train the AI with A/B comparisons")
print("   4. 👤 View your musical taste profile")
print("   5. 📊 Explore analytics and insights")


## 🎮 Step 7: Interactive Demo


In [ ]:
# Run the interactive demo
import asyncio
import sys
sys.path.append('/content/hack-mvp')

from demo.showcase import HackathonDemo

print("🎮 Running interactive demo...")
print("   This showcases all the features of the Music AI system")

demo = HackathonDemo()
await demo.run_full_demo()


## 🎉 Congratulations!

You've successfully set up and running the **AI Music Recommendation System** with:

- ✅ **GPT-OSS-20B** for natural language explanations
- ✅ **Neural Networks** for personalized recommendations
- ✅ **RLHF Training** with interactive A/B comparisons
- ✅ **Spotify Integration** with OAuth authentication
- ✅ **Real-time Learning** from user feedback
- ✅ **Beautiful UI** with analytics and insights

### 🎯 Next Steps:
1. **Authenticate with Spotify** in the app
2. **Get AI recommendations** for your playlists
3. **Train the AI** using A/B comparisons
4. **Explore your musical taste profile**
5. **Share your experience** with others!

### 🚀 Deploy to Production:
- Use the Docker setup for cloud deployment
- Scale with multiple GPU instances
- Add more AI models and features

**Enjoy your AI-powered music discovery! 🎵**
